In [1]:
import numpy as np
import scipy.io

In [2]:
data = scipy.io.loadmat('textdata.mat')

- Vocabulary is a V ×1 dimensional cell array that that contains every word appearing in the documents. When we refer to the jth word, we mean Vocabulary(j,1).
- XTrain is a n × V dimensional matrix describing the n documents used for training your Naive Bayes classifier. The entry XTrain(i,j) is 1 if word j appears in the ith training document and 0 otherwise.
- yTrain is a n×1 dimensional matrix containing the class labels for the training documents. yTrain(i,1) is 1 if the ith document belongs to The Economist and 2 if it belongs to The Onion.
- XTest and yTest are the same as XTrain and yTrain, except instead of having n rows, they have m rows. This is the data you will test your classifier on and it should not be used for training.

In [3]:
# check types
[print(k, type(data[k])) for k in data.keys()];

__header__ <class 'bytes'>
__version__ <class 'str'>
__globals__ <class 'list'>
XTrain <class 'scipy.sparse.csc.csc_matrix'>
XTest <class 'scipy.sparse.csc.csc_matrix'>
XTrainSmall <class 'scipy.sparse.csc.csc_matrix'>
yTrain <class 'numpy.ndarray'>
yTest <class 'numpy.ndarray'>
yTrainSmall <class 'numpy.ndarray'>
Vocabulary <class 'numpy.ndarray'>


In [4]:
# convert all to numpy arrays
splits = ['XTrain', 'XTest', 'XTrainSmall']
x_train, x_test, x_train_small = [data[s].toarray() for s in splits]
splits = ['yTrain', 'yTest', 'yTrainSmall', 'Vocabulary']
y_train, y_test, y_train_small, vocab = [data[s] for s in splits]
# Sanity check
print(x_train.shape[0] == y_train.shape[0])
print(x_test.shape[0] == y_test.shape[0])
print(x_train.shape[1] == x_test.shape[1])
print(vocab.shape[0] == x_test.shape[1])

True
True
True
True


<img src="assets/estimates.png" width="40%">

In [5]:
def log_prod(log_x):
    """
    Returns the product of elements in logspace.
    """
    return np.sum(log_x, axis=0)

# Check
print(log_prod([np.log(3), np.log(5)]) == np.log(3*5))


def xgiveny(x_train, y_train):
    """
     The output is a (2×Vocab), where for any word index w ∈ {1,...,Vocab}
     and class index y ∈ {1, 2}, the entry out(y,w) is the MAP estimate of 
     θ_yw = P(X_w = 1|Y = y) with a Beta(2,1) prior distribution.
    """
    mask_one = y_train == 1
    mask_two = y_train == 2
    out = np.ndarray((2, x_train.shape[1]))
    # See image above for formula
    out[0,:] = (np.sum(mask_one * x_train, axis=0) + 1) / ((np.sum(mask_one, axis=0)+ 1))
    out[1,:] = (np.sum(mask_two * x_train, axis=0) + 1) / ((np.sum(mask_two, axis=0)+ 1))
    return out


def yprior(y_train):
    """
    Returns prior, the estimation of P(θ_head=1)

    """
    out = np.ndarray(2, )
    return (np.sum(y_train==1, axis=0) / y_train.shape[0])[0]


def accuracy(preds, ground_truths):
    return np.sum(preds == ground_truths) / preds.shape[0] * 100


def classify(likelihood, y_prior, x_pred):
    """
    likelihood: (2, Vocab) 
    y_prior: value for yk=0
    """
    sample_size = x_pred.shape[0]
    classes = np.ndarray((sample_size, 1))
    for i in range(sample_size):
        # find P(X_pred | Y = 0)
        probs_0 = likelihood[0,:] * x_pred[i,:] + (1-likelihood[0,:]) * (1-x_pred[i, :])
        # find P(X_pred | Y = 1)
        probs_1 = likelihood[1,:] * x_pred[i,:] + (1-likelihood[1,:]) * (1-x_pred[i, :])
        # do logarithm trick, concat probs and prior
        score_0 = log_prod(np.concatenate(([np.log(prior)], np.log(probs_0))))
        score_1 = log_prod(np.concatenate(([np.log(1 - prior)], np.log(probs_1))))
        classes[i] = 1 if score_0 > score_1 else 2
    return classes

True


In [6]:
# Learn parameters from training data
likelihood = xgiveny(x_train, y_train)
prior = yprior(y_train)
# Since we have extracted likelihood and y_prior, we can make predictions
preds = classify(likelihood, prior, x_test)
accuracy(preds, y_test)

/Users/computa/miniconda3/envs/AI/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log


97.93103448275862

In [7]:
# Try estimating parameters from small dataset
likelihood = xgiveny(x_train_small, y_train_small)
prior = yprior(y_train_small)
# Since we have extracted likelihood and y_prior, we can make predictions
preds = classify(likelihood, prior, x_test)
accuracy(preds, y_test)

/Users/computa/miniconda3/envs/AI/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log


72.41379310344827